In [1]:
# Import packages
import pandas as pd
import ast 
import numpy as np
import json
import re

# Load data
with open('../data/tpb_tweets_news-outlets_20211208.json', "r") as outfile:
    all_data = json.load(outfile)

# Filter data based on regex    
regex = re.compile(r"\b#Pandemic\b|\b#Covid\b|\b#Covid-19\b|\b#migrants\b|\b#ETH\b|\b#Cardano\b|\b#refugees\b|\b#transit\b|\b#displacement\b|\b#borders\b|\b#return\b|\b#pushbacks\b|\b#boat\b|\b#drowning\b|\b#hunger\b|\b#refugee camp\b|\bPandemic\b|\bCovid\b|\bCovid-19\b|\bmigrants\b|\brefugees\b|\btransit\b|\bdisplacement\b|\bborders\b|\breturn\b|\bpushbacks\b|\bboat\b|\bdrowning\b|\bhunger\b|\brefugee camp\b", re.IGNORECASE)

data_filtered = []

for entry in all_data.get('data'):
    if regex.search(entry.get('text')):
        data_filtered.append(entry)


In [2]:
# Convert data to df

df_tweets = pd.DataFrame.from_records(data_filtered)
df_users = pd.DataFrame.from_records(all_data.get('includes').get('users'))

In [3]:
df_tweets.head()

,entities,public_metrics,created_at,text,id,author_id,referenced_tweets
0,"{'urls': [{'start': 121, 'end': 144, 'url': 'h...","{'retweet_count': 41, 'reply_count': 12, 'like...",2021-12-07T23:50:06.000Z,A U.S. judge on Tuesday blocked the Biden admi...,1468367242064760841,3108351,NaN
1,"{'urls': [{'start': 94, 'end': 117, 'url': 'ht...","{'retweet_count': 44, 'reply_count': 33, 'like...",2021-12-07T23:18:46.000Z,Analysis: The wide and dangerous gap between T...,1468359359080054784,2467791,NaN
2,"{'urls': [{'start': 162, 'end': 185, 'url': 'h...","{'retweet_count': 16, 'reply_count': 5, 'like_...",2021-12-07T23:00:47.000Z,Why are many pregnant women hesitant to take t...,1468354833719697427,4970411,NaN
3,"{'urls': [{'start': 173, 'end': 196, 'url': 'h...","{'retweet_count': 59, 'reply_count': 94, 'like...",2021-12-07T22:15:08.000Z,While much is still unknown about the Omicron ...,1468343345642618884,807095,NaN
4,"{'annotations': [{'start': 26, 'end': 40, 'pro...","{'retweet_count': 3, 'reply_count': 1, 'like_c...",2021-12-07T21:56:04.000Z,6/ What has played out in Lower Manhattan amon...,1468338544582316037,807095,"[{'type': 'replied_to', 'id': '146833854276188..."


In [4]:
df_users.head()

,description,public_metrics,verified,name,url,username,created_at,id
0,Democracy Dies in Darkness,"{'followers_count': 18516470, 'following_count...",True,The Washington Post,https://t.co/1KN78z0bbe,washingtonpost,2007-03-27T11:19:39.000Z,2467791
1,News tips? Share them here: https://t.co/ghL9O...,"{'followers_count': 51136117, 'following_count...",True,The New York Times,http://t.co/ahvuWqicF9,nytimes,2007-03-02T20:41:42.000Z,807095
2,Sign up for our newsletters and email alerts: ...,"{'followers_count': 19263346, 'following_count...",True,The Wall Street Journal,https://t.co/GhhR6PLfem,WSJ,2007-04-01T06:22:13.000Z,3108351
3,Hear the human story and join the discussion. ...,"{'followers_count': 7292176, 'following_count'...",True,Al Jazeera English,https://t.co/fDSF1GNVXl,AJEnglish,2007-04-17T08:23:08.000Z,4970411
4,Exposing #HumanRights abuses around the world....,"{'followers_count': 4817348, 'following_count'...",True,Human Rights Watch,https://t.co/7Dg4Sb42mK,hrw,2008-05-08T13:42:20.000Z,14700316


In [3]:
# Functions for unnesting

def fix_dicts(string):
    if string is np.nan:
        return(string)
    if not isinstance(string, dict):
        string_as_dict = ast.literal_eval(string)
        return(string_as_dict)
    else:
        return(string)

def unnest_hashtags(entities):
    try:
        hashtags = list(entities.get('hashtags'))
    except:
        return(list())
    if isinstance(hashtags, list):
        hashtags_list = [hashtag.get('tag') for hashtag in hashtags]
        return(hashtags_list)
    else:
        return
    
def unnest_mentions(entities):
    try:
        mentions = list(entities.get('mentions'))
    except:
        return(list())
    if isinstance(mentions, list):
        mentions_list = [mention.get('username') for mention in mentions]
        return(mentions_list)
    else:
        return

def unnest_urls(entities):
    try:
        urls = list(entities.get('urls'))
    except:
        return(list())
    if isinstance(urls, list):
        urls_list = [url.get('url') for url in urls]
        return(urls_list)
    else:
        return
    
def unnest_cashtags(entities):
    try:
        cashtags = list(entities.get('cashtags'))
    except:
        return(list())
    if isinstance(cashtags, list):
        cashtags_list = [cashtag.get('tag') for cashtag in cashtags]
        return(cashtags_list)
    else:
        return

In [4]:
# Fix dictionaries in df_tweets

df_tweets['public_metrics'] = df_tweets['public_metrics'].apply(fix_dicts)
df_tweets['entities'] = df_tweets['entities'].apply(fix_dicts)

df_tweets.head()

,entities,public_metrics,created_at,text,id,author_id,referenced_tweets
0,"{'urls': [{'start': 121, 'end': 144, 'url': 'h...","{'retweet_count': 41, 'reply_count': 12, 'like...",2021-12-07T23:50:06.000Z,A U.S. judge on Tuesday blocked the Biden admi...,1468367242064760841,3108351,NaN
1,"{'urls': [{'start': 94, 'end': 117, 'url': 'ht...","{'retweet_count': 44, 'reply_count': 33, 'like...",2021-12-07T23:18:46.000Z,Analysis: The wide and dangerous gap between T...,1468359359080054784,2467791,NaN
2,"{'urls': [{'start': 162, 'end': 185, 'url': 'h...","{'retweet_count': 16, 'reply_count': 5, 'like_...",2021-12-07T23:00:47.000Z,Why are many pregnant women hesitant to take t...,1468354833719697427,4970411,NaN
3,"{'urls': [{'start': 173, 'end': 196, 'url': 'h...","{'retweet_count': 59, 'reply_count': 94, 'like...",2021-12-07T22:15:08.000Z,While much is still unknown about the Omicron ...,1468343345642618884,807095,NaN
4,"{'annotations': [{'start': 26, 'end': 40, 'pro...","{'retweet_count': 3, 'reply_count': 1, 'like_c...",2021-12-07T21:56:04.000Z,6/ What has played out in Lower Manhattan amon...,1468338544582316037,807095,"[{'type': 'replied_to', 'id': '146833854276188..."


In [27]:
# Unnest columns

drop_cols = ['public_metrics', 'entities']

df_tweets_unnest = pd.concat([df_tweets, pd.json_normalize(df_tweets['public_metrics'])], axis = 1)
df_tweets_unnest['hashtags'] = df_tweets_unnest['entities'].apply(unnest_hashtags)
df_tweets_unnest['mentions'] = df_tweets_unnest['entities'].apply(unnest_mentions)
df_tweets_unnest['urls'] = df_tweets_unnest['entities'].apply(unnest_urls)
df_tweets_unnest['cashtags'] = df_tweets_unnest['entities'].apply(unnest_cashtags)
df_tweets_unnest = df_tweets_unnest.loc[:, ~df_tweets_unnest.columns.isin(drop_cols)]

df_tweets_unnest.head()

,created_at,text,id,author_id,referenced_tweets,retweet_count,reply_count,like_count,quote_count,hashtags,mentions,urls,cashtags
0,2021-12-07T23:50:06.000Z,A U.S. judge on Tuesday blocked the Biden admi...,1468367242064760841,3108351,NaN,41,12,111,3,[],[],[https://t.co/FBUE9wljht],[]
1,2021-12-07T23:18:46.000Z,Analysis: The wide and dangerous gap between T...,1468359359080054784,2467791,NaN,44,33,114,1,[],[],[https://t.co/RCHGUQENAY],[]
2,2021-12-07T23:00:47.000Z,Why are many pregnant women hesitant to take t...,1468354833719697427,4970411,NaN,16,5,34,1,[],[],[https://t.co/YlzjLNx0Cp],[]
3,2021-12-07T22:15:08.000Z,While much is still unknown about the Omicron ...,1468343345642618884,807095,NaN,59,94,192,7,[],[],[https://t.co/Zt4u7gpEZN],[]
4,2021-12-07T21:56:04.000Z,6/ What has played out in Lower Manhattan amon...,1468338544582316037,807095,"[{'type': 'replied_to', 'id': '146833854276188...",3,1,10,0,[],[],[],[]


In [28]:
# Fix dictionaries and unnest df_users

df_users['public_metrics'] = df_users['public_metrics'].apply(fix_dicts)
df_users_unnest = pd.concat([df_users, pd.json_normalize(df_users['public_metrics'])], axis = 1)
df_users_unnest = df_users_unnest.rename(columns = {'id': 'author_id', 'created_at': 'author_created_at'})
df_users_unnest = df_users_unnest.loc[:, ~df_users_unnest.columns.isin(drop_cols)]

df_users_unnest.head()

,description,verified,name,url,username,author_created_at,author_id,followers_count,following_count,tweet_count,listed_count
0,Democracy Dies in Darkness,True,The Washington Post,https://t.co/1KN78z0bbe,washingtonpost,2007-03-27T11:19:39.000Z,2467791,18516470,1715,412041,2
1,News tips? Share them here: https://t.co/ghL9O...,True,The New York Times,http://t.co/ahvuWqicF9,nytimes,2007-03-02T20:41:42.000Z,807095,51136117,860,455829,211884
2,Sign up for our newsletters and email alerts: ...,True,The Wall Street Journal,https://t.co/GhhR6PLfem,WSJ,2007-04-01T06:22:13.000Z,3108351,19263346,1046,357831,122904
3,Hear the human story and join the discussion. ...,True,Al Jazeera English,https://t.co/fDSF1GNVXl,AJEnglish,2007-04-17T08:23:08.000Z,4970411,7292176,238,293680,54442
4,Exposing #HumanRights abuses around the world....,True,Human Rights Watch,https://t.co/7Dg4Sb42mK,hrw,2008-05-08T13:42:20.000Z,14700316,4817348,21214,121343,22486


In [29]:
# Combine data with pd.merge

df_combined = pd.merge(df_tweets_unnest, df_users_unnest, how = 'left', left_on = 'author_id', right_on = 'author_id').drop_duplicates(subset = ['id'])
df_combined.head()

,created_at,text,id,author_id,referenced_tweets,retweet_count,reply_count,like_count,quote_count,hashtags,...,description,verified,name,url,username,author_created_at,followers_count,following_count,tweet_count,listed_count
0,2021-12-07T23:50:06.000Z,A U.S. judge on Tuesday blocked the Biden admi...,1468367242064760841,3108351,NaN,41,12,111,3,[],...,Sign up for our newsletters and email alerts: ...,True,The Wall Street Journal,https://t.co/GhhR6PLfem,WSJ,2007-04-01T06:22:13.000Z,19263346,1046,357831,122904
848,2021-12-07T23:18:46.000Z,Analysis: The wide and dangerous gap between T...,1468359359080054784,2467791,NaN,44,33,114,1,[],...,Democracy Dies in Darkness,True,The Washington Post,https://t.co/1KN78z0bbe,washingtonpost,2007-03-27T11:19:39.000Z,18516470,1715,412041,2
1632,2021-12-07T23:00:47.000Z,Why are many pregnant women hesitant to take t...,1468354833719697427,4970411,NaN,16,5,34,1,[],...,Hear the human story and join the discussion. ...,True,Al Jazeera English,https://t.co/fDSF1GNVXl,AJEnglish,2007-04-17T08:23:08.000Z,7292176,238,293680,54442
2387,2021-12-07T22:15:08.000Z,While much is still unknown about the Omicron ...,1468343345642618884,807095,NaN,59,94,192,7,[],...,News tips? Share them here: https://t.co/ghL9O...,True,The New York Times,http://t.co/ahvuWqicF9,nytimes,2007-03-02T20:41:42.000Z,51136117,860,455829,211884
3158,2021-12-07T21:56:04.000Z,6/ What has played out in Lower Manhattan amon...,1468338544582316037,807095,"[{'type': 'replied_to', 'id': '146833854276188...",3,1,10,0,[],...,News tips? Share them here: https://t.co/ghL9O...,True,The New York Times,http://t.co/ahvuWqicF9,nytimes,2007-03-02T20:41:42.000Z,51136117,860,455829,211884


In [12]:
df_combined.to_csv('tpb_df.csv', index=False)